<a href="https://colab.research.google.com/github/ruvenkotz/SAT-Solver-Sudoku/blob/main/SAT_Solver_for_sudoku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SAT solver for sudoku**

In [1]:
!pip install python-sat

     |████████████████████████████████| 1.8MB 33.2MB/s 


In [5]:
n = 9 

# we create a mapping from propositional symbol v_{x,(i, j)}
# to variables indices in SAT solver.
varmap = dict()
# we also store the reverse mapping from variables 
# indices in SAT solver to propositional symbol v_{x,(i, j)}
# so that we can later visualize the solution.
mapback = dict()

idx = 1
for i in range(n):
    for j in range(n):
        for x in range(1, n+1):
            varmap[(i, j, x)] = idx
            mapback[idx] = (i, j, x)
            idx += 1

In [6]:
import itertools

#[row] creates row constraints 
# You can't have the same number in a row

def row(solver, ivarmap, n):
    for x in range(1, n+1):
        for i in range(n):
            in_row = [ivarmap[(i,j,x)] for j in range(n)]
            # these are mutually exclusive constraints saying that
            # each symbol appears *at most* once in the row.
            for (c1, c2) in itertools.combinations(in_row, 2):
                # no any pairs of variables can be true simultaneously
                solver.add_clause([-c1, -c2])
            # this is an at least 1 constraint saying that there
            # has to be one symbol that appears in the row
            solver.add_clause(in_row)

#[column] creates column constraints 
# You can't have the same number in a column

def column(solver, ivarmap, n):
    for x in range(1, n+1):
        for j in range(n):
            in_column = [ivarmap[(i,j,x)] for i in range(n)]
            for (c1, c2) in itertools.combinations(in_column, 2):
                solver.add_clause([-c1, -c2])
            solver.add_clause(in_column)

#[square] creates blocks constraints 
# You can't have the same number in 3x3 box

def square(solver, ivarmap, n):
    sqrt_n = np.sqrt(n)
    assert sqrt_n == int(sqrt_n)
    sqrt_n = int(sqrt_n)
    for x in range(1, n+1):
        for i in range(0, n, sqrt_n):
            for j in range(0, n, sqrt_n):
                in_square = [ivarmap[(p,q,x)] 
                for p in range(i, i+sqrt_n) 
                for q in range(j, j+sqrt_n)]
                for (c1, c2) in itertools.combinations(in_square, 2):
                    solver.add_clause([-c1, -c2])
                solver.add_clause(in_square)

#[cell] creates cell constraints 
# You must have a number 1-9 in each cell

def cell(solver, ivarmap):
    for i in range(n):
        for j in range(n):
            in_cell = [ivarmap[(i,j,x)] for x in range(1, n+1)]
            for (c1, c2) in itertools.combinations(in_cell, 2):
                solver.add_clause([-c1, -c2])
            solver.add_clause(in_cell)

In [13]:
import numpy as np
from pysat.solvers import Minisat22


s = Minisat22()

# Adding the constraints
row(s, varmap, n)
column(s, varmap, n)
square(s, varmap, n)
cell(s, varmap)

# Defining the starting board
puzzle =  np.matrix([[0,7,0,0,0,0,0,0,0],
                     [0,8,0,4,0,0,0,7,0],
                     [0,0,0,0,5,3,0,4,0],
                     [0,0,9,1,0,0,7,3,0],
                     [7,0,0,0,0,2,0,0,0],
                     [0,0,0,0,0,4,9,0,2],
                     [5,0,0,0,0,9,8,0,0],
                     [0,0,4,5,0,0,0,0,0],
                     [1,0,3,0,2,0,0,0,4]])

# [start] adds each of the starting board numbers as a constraint 
def start(solver, ivarmap):
    for i in range(n):
        for j in range(n):
          for x in range(1,n+1):
            if puzzle[i,j] == x:
              c = ivarmap[(i,j,x)]
              solver.add_clause([c])


#[get_solution] outputs one possible solution 
# We have to figure out if the solution is unique
def get_solution(imodel, imapback):
  for i in range(len(imodel)):
    if imodel[i] > 0:
      x, y, val = imapback[i+1]
      puzzle[x, y] = val
  return puzzle 


#[new_solution] adds the solution as a constraint
# The new solution can't be the old solution
def new_solution(solver, ivarmap):
    for i in range(n):
        for j in range(n):
          for x in range(1,n+1):
            if solution[i,j] == x:
              c = ivarmap[(i,j,x)]
              solver.add_clause([-c])


s = Minisat22()
# We add the constraints
row(s, varmap, n)
column(s, varmap, n)
square(s, varmap, n)
cell(s, varmap)
start(s,varmap)
    
#Solve once to find a solution
cell(s, varmap)
s.solve()
model = s.get_model()
if(model != None):
  solution = get_solution(model, mapback)
  print(solution)


# We add the solution as a constraint 
# and see if there is a second possible solution
new_solution(s,varmap)
cell(s, varmap)
s.solve()
model_2 = s.get_model()
if(model_2 == None):
 print("There is no other solution")


[[4 7 5 2 1 8 3 6 9]
 [3 8 2 4 9 6 1 7 5]
 [9 1 6 7 5 3 2 4 8]
 [2 4 9 1 8 5 7 3 6]
 [7 3 8 9 6 2 4 5 1]
 [6 5 1 3 7 4 9 8 2]
 [5 2 7 6 4 9 8 1 3]
 [8 9 4 5 3 1 6 2 7]
 [1 6 3 8 2 7 5 9 4]]
There is no other solution
